In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
from api_keys import Client_ID, Client_Secret, Username, redirect_uri
import sys
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import json
from pprint import pprint

In [2]:
client_credentials_manager=SpotifyClientCredentials(client_id=Client_ID, client_secret=Client_Secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
token = util.prompt_for_user_token(username=Username,
                           scope=None,
                           client_id=Client_ID,
                           client_secret=Client_Secret,
                           redirect_uri=redirect_uri)
if token == token:
    print('It works')

it works


In [4]:
# This is to get Led Zeplin names, url, and image url
# lz_uri = 'spotify:artist:36QJpDe2go2KgaRleHCDTp'
# results = spotify.artist_top_tracks(lz_uri)

# for track in results['tracks'][:10]:
#     print('track    : ' + track['name'])
#     print('audio    : ' + track['preview_url'])
#     print('cover art: ' + track['album']['images'][0]['url'])
#     print()
# v = 'https://open.spotify.com/playlist/7AfWDGHu83rrBqvpg9reh9?si=XaBWARddRPGVfQBIvSJEvw'
# m = 'https://open.spotify.com/playlist/37i9dQZF1DX8CopunbDxgW?si=_cGN7PBeTrC51OmQYNg5pA'

In [5]:
# metropolis = 'spotify:playlist:37i9dQZF1DX8CopunbDxgW'
# metropolis = '37i9dQZF1DX8CopunbDxgW'# This I like
# vapourwave = 'spotify:playlist:7AfWDGHu83rrBqvpg9reh9' # This I do not like

In [6]:
# results_m = sp.audio_features(metropolis)

# print(results_m)